# Flash Evaluation on CamFow Dataset:

This notebook is dedicated to evaluating Flash on the 5G CamFlow datasets, which are graph-level in nature. We employ Flash in graph-level detection mode to analyze this dataset effectively. Upon completion of the notebook execution, the results will be presented.

## Dataset Access: 
- This dataset will be publically available upon publishing

## Data Parsing and Execution:
- Utilize the parser included in this notebook to process the downloaded files.
- To obtain the evaluation results, execute all cells within this notebook.

## Model Training and Execution Flexibility:
- By default, the notebook operates using pre-trained model weights.
- Additionally, this notebook offers the flexibility to set parameters for training Graph Neural Networks (GNNs) and word2vec models from scratch.
- You can then utilize these freshly trained models to conduct the evaluation. 

Follow these guidelines for a thorough and efficient analysis of the Unicorn datasets using Flash.


In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch_geometric.data import Data
import torch.nn.functional as F
import json
import warnings
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
warnings.filterwarnings('ignore')
from torch_geometric.loader import NeighborLoader
import multiprocessing
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
%matplotlib inline

In [2]:
from pprint import pprint
import gzip
from sklearn.manifold import TSNE
import json
import copy
import os
from tqdm import tqdm, trange

In [3]:
Train_Gnn = True
Train_Word2vec = False
Parse_data = False

In [4]:
from tqdm import tqdm 
def prepare_graph(df):
    def process_node(node, action, node_dict, label_dict, dummies, node_type):
        node_dict.setdefault(node, []).append(action)
        label_dict[node] = dummies.get(getattr(row, node_type), -1)  

    nodes = {}
    labels = {}
    edges = []
    # dummies = {
    #     "7998762093665332071": 0,
    #     "14709879154498484854": 1,
    #     "10991425273196493354": 2,
    #     "14871526952859113360": 3,
    #     "8771628573506871447": 4,
    #     "7877121489144997480": 5,
    #     "17841021884467483934": 6,
    #     "7895447931126725167": 7,
    #     "15125250455093594050": 8,
    #     "8664433583651064836": 9,
    #     "14377490526132269506": 10,
    #     "15554536683409451879": 11,
    #     "8204541918505434145": 12,
    #     "14356114695140920775": 13
    # }
    
    dummies = {'address': 0,
        'argv': 1,
        'block': 2,
        'file': 3,
        'iattr': 4,
        'link': 5,
        'path': 6,
        'pipe': 7,
        'process_memory': 8,
        'socket': 9,
        'task': 10,
        'xattr': 11}

    for row in df.itertuples():
        process_node(row.actorID, row.action, nodes, labels, dummies, 'actor_type')
        process_node(row.objectID, row.action, nodes, labels, dummies, 'object')

        edges.append((row.actorID, row.objectID))

    features = [nodes[node] for node in tqdm(nodes)]
    feat_labels = [labels[node] for node in tqdm(nodes)]
    edge_index = [[], []]
    node_index_map = {node: i for i, node in enumerate(nodes.keys())}
    for src, dst in tqdm(edges):
        src_index = node_index_map[src]
        dst_index = node_index_map[dst]
        edge_index[0].append(src_index)
        edge_index[1].append(dst_index)

    return features, feat_labels, edge_index, list(nodes.keys())


In [5]:
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv, GATConv
import torch.nn.functional as F
import torch.nn as nn

class GCN(torch.nn.Module):
    def __init__(self,in_channel,out_channel):
        super().__init__()
        self.conv1 = SAGEConv(in_channel, 32, normalize=True)
        self.conv2 = SAGEConv(32, 20, normalize=True)
        self.linear = nn.Linear(in_features=20,out_features=out_channel)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = self.linear(x)
        return F.softmax(x, dim=1)

In [6]:
from gensim.models.callbacks import CallbackAny2Vec
import gensim
from gensim.models import Word2Vec
from multiprocessing import Pool
from itertools import compress
from tqdm import tqdm


class EpochSaver(CallbackAny2Vec):
    '''Callback to save model after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        model.save('trained_weights/5gcamflow/5gcamflow.model')
        self.epoch += 1

In [7]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_begin(self, model):
        print("Epoch #{} start".format(self.epoch))

    def on_epoch_end(self, model):
        print("Epoch #{} end".format(self.epoch))
        self.epoch += 1

In [8]:
logger = EpochLogger()
saver = EpochSaver()

In [9]:
if Train_Word2vec:
    comb_data = []
    for i in tqdm(range(12)):
        f = open(f"5gcamflow/{i}.txt")
        data = f.read().split('\n')
        data = [line.split('\t') for line in data]
        comb_data = comb_data + data

    df = pd.DataFrame (comb_data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()
    phrases,labels,edges,mapp = prepare_graph(df)
    


In [10]:
if Train_Word2vec:
        word2vec = Word2Vec(sentences=phrases, vector_size=30, window=5, min_count=1, workers=32,epochs=300,callbacks=[saver,logger])

In [11]:
from sklearn.utils import class_weight
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss

model = GCN(30,14).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [12]:
from collections import Counter
import math

class PositionalEncoder:

    def __init__(self, d_model, max_len=100000, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        position = torch.arange(max_len, device=device).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2, device=device) * (-math.log(10000.0) / d_model))
        self.pe = torch.zeros(max_len, d_model, device=device)
        self.pe[:, 0::2] = torch.sin(position * div_term)
        self.pe[:, 1::2] = torch.cos(position * div_term)

    def embed(self, x):
        return x + self.pe[:x.size(0)]

def infer(document):
    word_embeddings = [w2vmodel.wv[word] for word in document if word in  w2vmodel.wv]
    
    if not word_embeddings:
        return np.zeros(30)

    output_embedding = torch.tensor(word_embeddings, dtype=torch.float, device=device)
    if len(document) < 100000:
        output_embedding = encoder.embed(output_embedding)

    output_embedding = output_embedding.detach().cpu().numpy()
    return np.mean(output_embedding, axis=0)

encoder = PositionalEncoder(30)
w2vmodel = Word2Vec.load("trained_weights/5gcamflow/5gcamflow.model")

In [13]:
from torch_geometric import utils

################################## Training Main Model #####################################
if Train_Gnn:
    for i in trange(299):
        f = open(f"5gcamflow/{i}.txt")
        data = f.read().split('\n')

        data = [line.split('\t') for line in data]
        df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
        df.sort_values(by='timestamp', ascending=True,inplace=True)
        df = df.dropna()
        phrases,labels,edges,mapp = prepare_graph(df)

        criterion = CrossEntropyLoss()

        nodes = [infer(x) for x in tqdm(phrases, desc='Inferring Phrases')]
        nodes = np.array(nodes)  

        graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
        graph.n_id = torch.arange(graph.num_nodes)
        mask = torch.tensor([True]*graph.num_nodes, dtype=torch.bool)

        for m_n in range(20):
            loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
            total_loss = 0
            for subg in loader:
                subg.to(device)
                model.train()
                optimizer.zero_grad() 
                out = model(subg.x, subg.edge_index) 
                loss = criterion(out, subg.y) 
                loss.backward() 
                optimizer.step()      
                total_loss += loss.item() * subg.batch_size

            loader = NeighborLoader(graph, num_neighbors=[-1,-1], batch_size=5000,input_nodes=mask)
            for subg in loader:
                subg.to(device)
                model.eval()
                out = model(subg.x, subg.edge_index)
                sorted, indices = out.sort(dim=1,descending=True)
                conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
                conf = (conf - conf.min()) / conf.max()
                pred = indices[:,0]
                cond = (pred == subg.y)
                mask[subg.n_id[cond]] = False

            print(f'Model# {m_n}. {mask.sum().item()} nodes still misclassified out of {mask.size()} \n')
            torch.save(model.state_dict(), f'trained_weights/5gcamflow/5gcamflow{m_n}.pth')

Inferring Phrases: 100%|██████████| 12742/12742 [00:00<00:00, 418609.08it/s]


Model# 0. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 1. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 2. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 3. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 4. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 5. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 6. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 7. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 8. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 9. 5977 nodes still misclassified out of torch.Size([12742]) 



  0%|          | 1/299 [00:01<06:37,  1.33s/it]

Model# 10. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 11. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 12. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 13. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 14. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 15. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 16. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 17. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 18. 5977 nodes still misclassified out of torch.Size([12742]) 

Model# 19. 5977 nodes still misclassified out of torch.Size([12742]) 



Inferring Phrases: 100%|██████████| 13248/13248 [00:00<00:00, 324777.25it/s]


...

Model# 19. 4182 nodes still misclassified out of torch.Size([13483]) 



Inferring Phrases: 100%|██████████| 13422/13422 [00:00<00:00, 391803.88it/s]


Model# 0. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 1. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 2. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 3. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 4. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 5. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 6. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 7. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 8. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 9. 5056 nodes still misclassified out of torch.Size([13422]) 



100%|██████████| 299/299 [02:56<00:00,  1.69it/s]

Model# 10. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 11. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 12. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 13. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 14. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 15. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 16. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 17. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 18. 5056 nodes still misclassified out of torch.Size([13422]) 

Model# 19. 5056 nodes still misclassified out of torch.Size([13422]) 



### Validation

In [14]:
from torch.utils.data import Dataset, DataLoader
class PhraseDataset(Dataset):
    def __init__(self, phrases):
        self.phrases = phrases

    def __len__(self):
        return len(self.phrases)

    def __getitem__(self, idx):
        return self.phrases[idx]

def batch_infer(phrases):
    return [infer(phrase) for phrase in phrases] 

In [15]:
from tqdm import tqdm

for i in range(300, 319):
    print(f"Graph #: {i}")
    f = open(f"5gcamflow/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()

    print("Preparing Graph")
    phrases,labels,edges,mapp = prepare_graph(df)
    print("Done")
    print("Inferring words")
    # dataset = PhraseDataset(phrases)
    # loader = DataLoader(dataset, batch_size=10, num_workers=4)
    # nodes = []
    # for batch in tqdm(loader):
    #     batch_embeddings = batch_infer(batch)
    #     nodes.extend(batch_embeddings)
    nodes = [infer(x) for x in tqdm(phrases)]
    nodes = np.array(nodes)  
    print("Done!")

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)
    print("Evaluating...")
    for m_n in tqdm(range(20)):
        model.load_state_dict(torch.load(f'trained_weights/5gcamflow/5gcamflow{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y).to(device)
        falses = torch.tensor([False] * len(flag[graph.n_id[cond]]), dtype=torch.bool).to(device)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], falses)
    print("Done!")
            
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)

Graph #: 300
Preparing Graph


100%|██████████| 30000/30000 [00:00<00:00, 2706410.00it/s]


Done
Inferring words


100%|██████████| 12516/12516 [00:00<00:00, 375195.90it/s]


Done!
Evaluating...


...

Done
Inferring words


100%|██████████| 14744/14744 [00:00<00:00, 468391.69it/s]


Done!
Evaluating...


100%|██████████| 20/20 [00:00<00:00, 258.01it/s]

Done!
5805 39.37194791101465


### Testing

In [20]:
thresh = 5700

In [21]:
correct_benign = 0

for i in range(320,350):
    print(f"Graph #: {i}")
    f = open(f"5gcamflow/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()

    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in tqdm(phrases)]
    nodes = np.array(nodes)  

    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)

    for m_n in range(20):
        model.load_state_dict(torch.load(f'trained_weights/5gcamflow/5gcamflow{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y).to(device)
        falses = torch.tensor([False]*len(flag[graph.n_id[cond]]), dtype=torch.bool).to(device)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], falses)

    if flag.sum().item() <= thresh:
        correct_benign = correct_benign + 1
            
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)
print(correct_benign)

Graph #: 320


100%|██████████| 13728/13728 [00:00<00:00, 267481.50it/s]


5114 37.252331002331005
Graph #: 321


100%|██████████| 13790/13790 [00:00<00:00, 320683.58it/s]


5783 41.9361856417694
Graph #: 322


...

5143 37.913748617766316
Graph #: 348


100%|██████████| 13524/13524 [00:00<00:00, 445917.03it/s]


4917 36.357586512866014
Graph #: 349


100%|██████████| 13203/13203 [00:00<00:00, 428445.17it/s]

4735 35.863061425433614
27


In [24]:
correct_attack = 0

for i in range(400,430):
    print(f"Graph #: {i}")
    f = open(f"5gcamflow/{i}.txt")
    data = f.read().split('\n')

    data = [line.split('\t') for line in data]
    df = pd.DataFrame (data, columns = ['actorID', 'actor_type','objectID','object','action','timestamp'])
    df.sort_values(by='timestamp', ascending=True,inplace=True)
    df = df.dropna()
    
    phrases,labels,edges,mapp = prepare_graph(df)

    nodes = [infer(x) for x in phrases]
    nodes = np.array(nodes)  
    
    graph = Data(x=torch.tensor(nodes,dtype=torch.float).to(device),y=torch.tensor(labels,dtype=torch.long).to(device), edge_index=torch.tensor(edges,dtype=torch.long).to(device))
    graph.n_id = torch.arange(graph.num_nodes).to(device)
    flag = torch.tensor([True]*graph.num_nodes, dtype=torch.bool).to(device)

    for m_n in range(20):
        model.load_state_dict(torch.load(f'trained_weights/5gcamflow/5gcamflow{m_n}.pth'))
        model.eval()
        out = model(graph.x, graph.edge_index)

        sorted, indices = out.sort(dim=1,descending=True)
        conf = (sorted[:,0] - sorted[:,1]) / sorted[:,0]
        conf = (conf - conf.min()) / conf.max()

        pred = indices[:,0]
        cond = (pred == graph.y).to(device)
        falses = torch.tensor([False]*len(flag[graph.n_id[cond]]), dtype=torch.bool).to(device)
        flag[graph.n_id[cond]] = torch.logical_and(flag[graph.n_id[cond]], falses)

    if  flag.sum().item() > thresh:
        correct_attack = correct_attack + 1
   
    print(flag.sum().item(), (flag.sum().item() / len(flag))*100)

Graph #: 400


100%|██████████| 30000/30000 [00:00<00:00, 2127612.32it/s]


15797 100.0
Graph #: 401


100%|██████████| 30000/30000 [00:00<00:00, 2126857.10it/s]


15815 100.0
Graph #: 402


...

16528 100.0
Graph #: 428


100%|██████████| 30000/30000 [00:00<00:00, 2446419.10it/s]


15680 100.0
Graph #: 429


100%|██████████| 30000/30000 [00:00<00:00, 2408673.81it/s]


15036 100.0


In [26]:
TP = correct_attack
FP = 29 - correct_benign
TN = correct_benign
FN = 30 - correct_attack

FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
TPR = TP / (TP + FN) if (TP + FN) > 0 else 0

print(f"Number of True Positives (TP): {TP}")
print(f"Number of False Positives (FP): {FP}")
print(f"Number of False Negatives (FN): {FN}")
print(f"Number of True Negatives (TN): {TN}\n")

precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TPR  
print(f"Precision: {precision}")
print(f"Recall: {recall}")

fscore = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
print(f"Fscore: {fscore}\n")

Number of True Positives (TP): 30
Number of False Positives (FP): 2
Number of False Negatives (FN): 0
Number of True Negatives (TN): 27

Precision: 0.9375
Recall: 1.0
Fscore: 0.967741935483871

